In [1]:
import requests
import json
import csv
import datetime
from time import sleep
import numpy as np
import csv

def write_csv(filename,new_line):
    with open(filename, "a" ) as fout:
        writer=csv.writer(fout)
        writer.writerow((new_line))
        

fut = "XBTZ17"
spot = "XBTUSD"
pos = 20000
#pos_test = pos+pos*0.1
fut_commis = 0.000750
spot_commis = 0.000750
period = 15
period_bol = 1



timeframe = 60  # в секундах
cycle = 2
k=0

DATA = []
BOL = []

In [7]:
def pars (fut, spot, pos, fut_commis, spot_commis):
    
    # Запрос стакана по фьючерсу
    fut_url="https://www.bitmex.com/api/v1/orderBook?symbol="+fut+"&depth=25"
    fut_html=requests.get(fut_url).text
    fut_data=json.loads(fut_html)

    # Запрос стакана по споту
    spot_url="https://www.bitmex.com/api/v1/orderBook?symbol="+spot+"&depth=25"
    spot_html=requests.get(spot_url).text
    spot_data=json.loads(spot_html)

    # Получаем данные о дате поставки фьючерса и количество дней до экспирации
    fut_setl_url="https://www.bitmex.com/api/v1/instrument?symbol="+fut+"&count=2&reverse=true"
    fut_setl_html=requests.get(fut_setl_url).text
    fut_setl_data=json.loads(fut_setl_html)
    
    YYYY = int(fut_setl_data[0]['settle'][:4])
    MM = int(fut_setl_data[0]['settle'][5:7])
    DD = int(fut_setl_data[0]['settle'][8:10])

    Settle_Date = datetime.date(YYYY,MM,DD)
    Curr_Date = datetime.datetime.utcnow().date()
    Curr_Datetime = datetime.datetime.strftime(datetime.datetime.utcnow(), "%Y.%m.%d %H:%M:%S")
    Days_to_settle = Settle_Date - Curr_Date
    Days_to_settle = int(str(Days_to_settle)[:2])
    Days_for_funding = str(Days_to_settle*3)
        
            
    # Получаем взвешенные по объему позиции цены
    pos_test = pos+pos*0.1
        
    q=0
    w=[]
    e=[]
    for i in fut_data:           
        q=q+i["askSize"]    
        if q>pos_test:
            w.append([i["askSize"], i["askPrice"]])
        if q<pos_test:
            e.append([i["askSize"], i["askPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    fut_ask=round(r/pos_test)   


    q=0
    w=[]
    e=[]
    for i in fut_data:           
        q=q+i["bidSize"]    
        if q>pos_test:
            w.append([i["bidSize"], i["bidPrice"]])
        if q<pos_test:
            e.append([i["bidSize"], i["bidPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    fut_bid=round(r/pos_test)
    

    q=0
    w=[]
    e=[]
    for i in spot_data:           
        q=q+i["askSize"]    
        if q>pos_test:
            w.append([i["askSize"], i["askPrice"]])
        if q<pos_test:
            e.append([i["askSize"], i["askPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    spot_ask=round(r/pos_test)


    q=0
    w=[]
    e=[]
    for i in spot_data:           
        q=q+i["bidSize"]    
        if q>pos_test:
            w.append([i["bidSize"], i["bidPrice"]])
        if q<pos_test:
            e.append([i["bidSize"], i["bidPrice"]])      
    r=0
    t=0
    for j in e:
        r=j[0]*j[1]+r
        t=j[0]+t      
    r=w[0][1]*(pos_test-t)+r
    spot_bid=round(r/pos_test)
        
    # Расчет фондирования по свопу        
    funding_url="https://www.bitmex.com/api/v1/funding?symbol="+spot+"&count="+Days_for_funding+"&reverse=true"
    funding_html=requests.get(funding_url).text
    funding_data=json.loads(funding_html)
        
    funding_rate = []
    for i in funding_data:
        funding_rate.append(i["fundingRate"])
        
    funding_rate_settle = sum(funding_rate)*3
        
        # Расчет спредов (Спред = Фьючерс - СПОТ, значит при "+" спреде - продаем спред (продаем фьючерс, покупаем СПОТ)
        # при "-" - покупаем.)
        
    Spread_Buy = round(fut_ask-spot_bid-(fut_ask*fut_commis)-(spot_bid*spot_commis)+(spot_bid*funding_rate_settle),2)
    Spread_Sell = round(fut_bid-spot_ask-(fut_bid*fut_commis)-(spot_ask*spot_commis)-(spot_ask*funding_rate_settle),2)
    
    Spread_Buy_pct = round((((fut_ask-(fut_ask*fut_commis))/(spot_bid+(spot_bid*spot_commis)-(spot_bid*funding_rate_settle)))
                            **(1/float(Days_to_settle))-1)*36500,4)
    Spread_Sell_pct =round((((fut_bid+(fut_bid*fut_commis))/(spot_ask-(spot_ask*spot_commis)+(spot_ask*funding_rate_settle)))
                                **(1/float(Days_to_settle))-1)*36500,4)                 
          
        # Записываем данные в массив
        
    data = [Curr_Datetime, Days_to_settle, "fut_ask", fut_ask, "fut_bid", fut_bid, "spot_ask", spot_ask, "spot_bid",
            spot_bid, "Sp_Buy", Spread_Buy, "Sp_Sell", Spread_Sell, "Sp_Buy_pct", Spread_Buy_pct,
            "Sp_Sell_pct", Spread_Sell_pct, funding_rate_settle]
    
    return data        

In [8]:
def indicator (DATA, period, period_bol):   
    
    if len(zip(*DATA)[15]) < period: 
        std_buy = 0.0
        Bol_UP_buy = zip(*DATA)[15][len(DATA)-1]
        Bol_D_buy = zip(*DATA)[15][len(DATA)-1]
            
        std_sell = 0.0
        Bol_UP_sell = zip(*DATA)[17][len(DATA)-1]
        Bol_D_sell = zip(*DATA)[17][len(DATA)-1]
        
        #signal = 0
    else:
        std_buy = round(np.std(zip(*DATA)[15][-period:], ddof=1),4)
        Bol_UP_buy = (std_buy*period_bol) + zip(*DATA)[15][len(DATA)-1]
        Bol_D_buy = zip(*DATA)[15][len(DATA)-1] - (std_buy*period_bol)
            
        std_sell = round(np.std(zip(*DATA)[17][-period:], ddof=1),4)
        Bol_UP_sell = (std_sell*period_bol) + zip(*DATA)[17][len(DATA)-1]
        Bol_D_sell = zip(*DATA)[17][len(DATA)-1] - (std_sell*period_bol)        
        
    data = [DATA[-1][0], "Bol_D_buy", Bol_D_buy, "std_buy", std_buy, "Bol_UP_buy", Bol_UP_buy, "Bol_D_sell", Bol_D_sell,
            "std_sell", std_sell, "Bol_UP_sell", Bol_UP_sell]
    return data

In [9]:
def signal (DATA, BOL, period):
    
    signal = 0
    if len(zip(*DATA)[15]) >= period:        
        if len(zip(*DATA)[15]) >= period*1.5:
            if zip(*DATA)[15][len(DATA)-1] < zip(*BOL)[2][len(BOL)-2]: signal = "Buy_Spread"           
            if zip(*DATA)[17][len(DATA)-1] > zip(*BOL)[12][len(BOL)-2]: signal = "Sell_Spread"
         
    data = [DATA[-1][0], signal]
    return data

In [10]:
S = []
filename = "arb.txt"

while k < cycle:
    try:
        print k+1
        print pars(fut, spot, pos, fut_commis, spot_commis)
        DATA.append(pars(fut, spot, pos, fut_commis, spot_commis))
    
        print indicator(DATA, period, period_bol)
        BOL.append(indicator(DATA, period, period_bol))
    
        print signal (DATA, BOL, period)
        S.append(signal (DATA, BOL, period))
        
        data = [DATA, BOL, S]        
        write_csv(filename, data)
        
        sleep(timeframe)
        k=k+1
        
    except:
        print "fail"
        pass

1
['2017.11.24 15:49:48', 35, 'fut_ask', 8159.0, 'fut_bid', 8152.0, 'spot_ask', 8191.0, 'spot_bid', 8190.0, 'Sp_Buy', -458.37, 'Sp_Sell', 363.9, 'Sp_Buy_pct', -56.9981, 'Sp_Sell_pct', 50.9081, -0.05068500000000001]
['2017.11.24 15:49:49', 'Bol_D_buy', -56.9981, 'std_buy', 0.0, 'Bol_UP_buy', -56.9981, 'Bol_D_sell', 50.9081, 'std_sell', 0.0, 'Bol_UP_sell', 50.9081]
['2017.11.24 15:49:49', 0]
2
['2017.11.24 15:50:51', 35, 'fut_ask', 8154.0, 'fut_bid', 8150.0, 'spot_ask', 8190.0, 'spot_bid', 8190.0, 'Sp_Buy', -463.37, 'Sp_Sell', 362.86, 'Sp_Buy_pct', -57.6364, 'Sp_Sell_pct', 50.7794, -0.05068500000000001]
['2017.11.24 15:50:53', 'Bol_D_buy', -57.1278, 'std_buy', 0.0, 'Bol_UP_buy', -57.1278, 'Bol_D_sell', 50.6493, 'std_sell', 0.0, 'Bol_UP_sell', 50.6493]
['2017.11.24 15:50:53', 0]
3
['2017.11.24 15:52:09', 35, 'fut_ask', 8145.0, 'fut_bid', 8141.0, 'spot_ask', 8176.0, 'spot_bid', 8176.0, 'Sp_Buy', -457.64, 'Sp_Sell', 367.16, 'Sp_Buy_pct', -57.0049, 'Sp_Sell_pct', 51.4122, -0.0506850000000000

['2017.11.24 16:16:07', 'Bol_D_buy', -58.9561, 'std_buy', 0.4161, 'Bol_UP_buy', -58.1239, 'Bol_D_sell', 49.4168, 'std_sell', 0.3272, 'Bol_UP_sell', 50.0712]
['2017.11.24 16:16:07', 0]
22
['2017.11.24 16:17:10', 35, 'fut_ask', 8128.0, 'fut_bid', 8126.0, 'spot_ask', 8175.0, 'spot_bid', 8175.0, 'Sp_Buy', -473.58, 'Sp_Sell', 353.12, 'Sp_Buy_pct', -59.053, 'Sp_Sell_pct', 49.614, -0.05068500000000001]
['2017.11.24 16:17:12', 'Bol_D_buy', -59.5117, 'std_buy', 0.4587, 'Bol_UP_buy', -58.5943, 'Bol_D_sell', 49.268499999999996, 'std_sell', 0.3455, 'Bol_UP_sell', 49.9595]
['2017.11.24 16:17:12', 0]
23
['2017.11.24 16:18:13', 35, 'fut_ask', 8127.0, 'fut_bid', 8125.0, 'spot_ask', 8174.0, 'spot_bid', 8174.0, 'Sp_Buy', -473.52, 'Sp_Sell', 353.07, 'Sp_Buy_pct', -59.0537, 'Sp_Sell_pct', 49.6132, -0.05068500000000001]
['2017.11.24 16:18:15', 'Bol_D_buy', -59.5606, 'std_buy', 0.5069, 'Bol_UP_buy', -58.5468, 'Bol_D_sell', 49.2466, 'std_sell', 0.3666, 'Bol_UP_sell', 49.9798]
['2017.11.24 16:18:15', 0]
24
['

In [ ]:
while k < cycle:
    try:
        # Запрос стакана по фьючерсу
        fut_url="https://www.bitmex.com/api/v1/orderBook?symbol="+fut+"&depth=25"
        fut_html=requests.get(fut_url).text
        fut_data=json.loads(fut_html)

        # Запрос стакана по споту
        spot_url="https://www.bitmex.com/api/v1/orderBook?symbol="+spot+"&depth=25"
        spot_html=requests.get(spot_url).text
        spot_data=json.loads(spot_html)

        # Получаем данные о дате поставки фьючерса и количество дней до экспирации
        fut_setl_url="https://www.bitmex.com/api/v1/instrument?symbol="+fut+"&count=2&reverse=true"
        fut_setl_html=requests.get(fut_setl_url).text
        fut_setl_data=json.loads(fut_setl_html)
    
        YYYY = int(fut_setl_data[0]['settle'][:4])
        MM = int(fut_setl_data[0]['settle'][5:7])
        DD = int(fut_setl_data[0]['settle'][8:10])

        Settle_Date = datetime.date(YYYY,MM,DD)
        Curr_Date = datetime.datetime.utcnow().date()
        Curr_Datetime = datetime.datetime.strftime(datetime.datetime.utcnow(), "%Y.%m.%d %H:%M:%S")
        Days_to_settle = Settle_Date - Curr_Date
        Days_to_settle = int(str(Days_to_settle)[:2])
        Days_for_funding = str(Days_to_settle*3)
        
            
        # Получаем взвешенные по объему позиции цены
        pos_test = pos+pos*0.1
        
        q=0
        w=[]
        e=[]
        for i in fut_data:           
            q=q+i["askSize"]    
            if q>pos_test:
                w.append([i["askSize"], i["askPrice"]])
            if q<pos_test:
                e.append([i["askSize"], i["askPrice"]])      
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[0]+t      
        r=w[0][1]*(pos_test-t)+r
        fut_ask=round(r/pos_test)   


        q=0
        w=[]
        e=[]
        for i in fut_data:           
            q=q+i["bidSize"]    
            if q>pos_test:
                w.append([i["bidSize"], i["bidPrice"]])
            if q<pos_test:
                e.append([i["bidSize"], i["bidPrice"]])      
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[0]+t      
        r=w[0][1]*(pos_test-t)+r
        fut_bid=round(r/pos_test)
    

        q=0
        w=[]
        e=[]
        for i in spot_data:           
            q=q+i["askSize"]    
            if q>pos_test:
                w.append([i["askSize"], i["askPrice"]])
            if q<pos_test:
                e.append([i["askSize"], i["askPrice"]])      
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[0]+t      
        r=w[0][1]*(pos_test-t)+r
        spot_ask=round(r/pos_test)


        q=0
        w=[]
        e=[]
        for i in spot_data:           
            q=q+i["bidSize"]    
            if q>pos_test:
                w.append([i["bidSize"], i["bidPrice"]])
            if q<pos_test:
                e.append([i["bidSize"], i["bidPrice"]])      
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[0]+t      
        r=w[0][1]*(pos_test-t)+r
        spot_bid=round(r/pos_test)
        
        # Расчет фондирования по свопу        
        funding_url="https://www.bitmex.com/api/v1/funding?symbol="+spot+"&count="+Days_for_funding+"&reverse=true"
        funding_html=requests.get(funding_url).text
        funding_data=json.loads(funding_html)
        
        funding_rate = []
        for i in funding_data:
            funding_rate.append(i["fundingRate"])
        
        funding_rate_settle = sum(funding_rate)*3
        
        # Расчет спредов (Спред = Фьючерс - СПОТ, значит при "+" спреде - продаем спред (продаем фьючерс, покупаем СПОТ)
        # при "-" - покупаем.)
        
        Spread_Buy = round(fut_ask-spot_bid-(fut_ask*fut_commis)-(spot_bid*spot_commis)+(spot_bid*funding_rate_settle),2)
        Spread_Sell = round(fut_bid-spot_ask-(fut_bid*fut_commis)-(spot_ask*spot_commis)-(spot_ask*funding_rate_settle),2)
    
        Spread_Buy_pct = round((((fut_ask-(fut_ask*fut_commis))/(spot_bid+(spot_bid*spot_commis)-(spot_bid*funding_rate_settle)))
                               **(1/float(Days_to_settle))-1)*36500,4)
        Spread_Sell_pct =round((((fut_bid+(fut_bid*fut_commis))/(spot_ask-(spot_ask*spot_commis)+(spot_ask*funding_rate_settle)))
                                 **(1/float(Days_to_settle))-1)*36500,4)                 
          
        # Записываем данные в массив
        
        data = [Curr_Datetime, Days_to_settle, "fut_ask", fut_ask, "fut_bid", fut_bid, "spot_ask", spot_ask, "spot_bid",
                 spot_bid, "Sp_Buy", Spread_Buy, "Sp_Sell", Spread_Sell, "Sp_Buy_pct", Spread_Buy_pct,
                 "Sp_Sell_pct", Spread_Sell_pct, "Bol_D_buy", Bol_D_buy, "std_buy", std_buy, "Bol_UP_buy", Bol_UP_buy,
                "Bol_D_sell", Bol_D_sell, "std_sell", std_sell, "Bol_UP_sell", Bol_UP_sell, "Signal", signal]
        DATA.append(data)
        
        print "len DATA", len(zip(*DATA)[15])
        print "period", period
        print "last line:", DATA[-1]
        print "spread:", zip(*DATA)[15], zip(*DATA)[17]     
        
        #считаем и записываем в массив стандартное отклонение
        if len(zip(*DATA)[15]) < period: 
            std_buy = 0.0
            Bol_UP_buy = 0.0
            Bol_D_buy = 0.0
            
            std_sell = 0.0
            Bol_UP_sell = 0.0
            Bol_D_sell = 0.0
        else:
            std_buy = round(np.std(zip(*DATA)[15][-period:], ddof=1),4)
            Bol_UP_buy = (std_buy*period_bol) + Spread_Buy_pct
            Bol_D_buy = Spread_Buy_pct - (std_buy*period_bol)
            
            std_sell = round(np.std(zip(*DATA)[17][-period:], ddof=1),4)
            Bol_UP_sell = (std_sell*period_bol) + Spread_Sell_pct
            Bol_D_sell = Spread_Sell_pct - (std_sell*period_bol)
            
            # Сигнал на вход в позицию
        
            if Spread_Buy_pct < Bol_D_buy: signal = "Buy_Spread"
            if Spread_Sell_pct > Bol_D_sell: signal = "Sell_Spread"
       
        print "bol_buy+ std_buy Bol_buy-:", Bol_D_buy, std_buy, Bol_UP_buy, Spread_Buy_pct
        print "bol_sell+ std_sell Bol_sell-:", Bol_D_sell, std_sell, Bol_UP_sell, Spread_Sell_pct
        print "___________________________________________"
        
        
        sleep(timeframe)
        k=k+1
        
    except:
        print "fail"
        pass

fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


In [13]:
print zip(*DATA)[15], zip(*DATA)[15][len(DATA)-1]
print zip(*BOL)[2], zip(*BOL)[2][len(BOL)-2]

(-11.7282, -11.7282, -11.849, -11.7279, -11.7279, -11.849, -11.7279, -11.7279, -12.3321, -12.3321) -12.3321
(0.0, 0.0, 0.0, 0.0, -11.782, -11.9153, -11.7942, -11.7821, -12.594100000000001, -12.6449) -12.5941


In [5]:
# Получаем данные о дате поставки фьючерса и количество дней до экспирации
fut_setl_url="https://www.bitmex.com/api/v1/instrument?symbol="+fut+"&count=2&reverse=true"
fut_setl_html=requests.get(fut_setl_url).text
fut_setl_data=json.loads(fut_setl_html)
    
YYYY = int(fut_setl_data[0]['settle'][:4])
MM = int(fut_setl_data[0]['settle'][5:7])
DD = int(fut_setl_data[0]['settle'][8:10])

Settle_Date = datetime.date(YYYY,MM,DD)
Curr_Date = datetime.datetime.utcnow().date()
Curr_Datetime = datetime.datetime.strftime(datetime.datetime.utcnow(), "%Y.%m.%d %H:%M:%S")
Cur_D = datetime.datetime.strftime(datetime.datetime.utcnow(), "%Y.%m.%d")
Cur_T = datetime.datetime.strftime(datetime.datetime.utcnow(), "%H:%M:%S")
Days_to_settle = Settle_Date - Curr_Date
Days_to_settle = int(str(Days_to_settle)[:2])
Days_for_funding = str(Days_to_settle*3)

In [6]:
print Cur_D
print Cur_T

2017.11.24
21:19:33
